# Neural Network Testing Framework
This notebook is intended to be a general framework for testing Neural Neworks on the EEG data set. While the initial iteration of this notebook is used for a simple Convolutional Neural Network (CNN), other networks using PyTorch can be implemented and assigned to the neural network variable. Many of the beginning cells are used to intialize for a Google Drive Colaboratory notebook GPU usability and can be ignored as necessary.

---



##Initialization
Google Drive access, PyTorch, etc.

In [0]:
!kill -9 -1

This section provides access to the user's Google drive.

In [1]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.


This section creates a drive that links to the user's Google drive.

In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

This section installs PyTorch.

In [0]:
# http://pytorch.org/
from os import path
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())

accelerator = 'cu80' if path.exists('/opt/bin/nvidia-smi') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.3.0.post4-{platform}-linux_x86_64.whl torchvision

##Imports

Imports can be added as necessary.

In [0]:
import numpy as np
import h5py
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import torch.cuda
from torch.utils.data import Dataset
from torch.autograd import Variable

##Classes

###EEGDataset
This class inherits the torch.utils.data.Dataset class to be used with the torch.utils.data.Dataloader class.

In [0]:
class EEGDataset(Dataset):
  """EEG dataset."""
  
  def __init__(self, x, y, transform=None):
    """
    Args:
      x (numpy array): Input data of shape 
                       num_trials x num_electrodes x num_time_bins.
      y (numpy array): Output data of shape num_trials x 1.
      transform (callable, optional): Optional transform to be applied.
    """
    self.x = x
    self.y = y
    self.transform = transform
    
  def __len__(self):
    return len(self.x)
  
  def __getitem__(self, idx):
    x_sample = torch.from_numpy(self.x[idx])
    y_sample = torch.IntTensor([int(self.y[idx])])
    
    if self.transform:
      pass #FIXME
    
    return x_sample, y_sample

###EEGMinimalContainer
This class holds a train and test EEGDataset. It processes the data into a (N, C, H, W) format in time.

In [0]:
class EEGMinimalContainer():
  """EEG container for training and testing datasets."""
  
  def __init__(self, data_dir, train_subject=None, test_subject=None, 
               remove_eog_channels=True, seed=42):
    """
    Args:
      data_dir (string): Path to all A0iT_slice.mat files for i in [1, 9].
      train_subject(int): Subject to train on. If None, train on all.
      test_subject(int): Subject to test on. If None, train on all except for
                         train_subject. Only used if train_subject is not None.
    """
    if train_subject is None:
      # Step 1: Append all of the input and output data together
      X = None
      y = None
      for i in range(1, 10):
        A0iT = h5py.File(data_dir + ('/A0%dT_slice.mat' % i), 'r')
        if X is None:
          X = np.copy(A0iT['image'])
          y = np.copy(A0iT['type'])
          y = y[0,0:X.shape[0]:1]
          y = np.asarray(y, dtype=np.int32)
        else:
          X_temp = np.copy(A0iT['image'])
          y_temp = np.copy(A0iT['type'])
          y_temp = y_temp[0,0:X_temp.shape[0]:1]
          y_temp = np.asarray(y_temp, dtype=np.int32)
          X = np.append(X, X_temp, axis=0)
          y = np.append(y, y_temp, axis=0)
      X = np.expand_dims(X, axis=1)
      y -= 769
      # Step 2: Remove the EOG
      if remove_eog_channels:
        X = X[:, :, 0:22, :] 
      # Step 3: Remove NaN trials
      remove_list = []
      for i in range(len(X)):
        if np.isnan(X[i]).any():
          remove_list.append(i)
      X = np.delete(X, remove_list, axis=0)
      y = np.delete(y, remove_list, axis=0)
      # Step 4: Generate an 82.43/17.57 train/test split
      X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                          test_size=0.1757, 
                                                          random_state=seed)
      print('EEGContainer X_train: ' + str(X_train.shape))
      print('EEGContainer y_train: ' + str(y_train.shape))
      print('EEGContainer X_test:  ' + str(X_test.shape))
      print('EEGContainer y_test:  ' + str(y_test.shape))
    
    else:
      pass #FIXME
    
    self.X_train = X_train
    self.y_train = y_train
    self.X_test = X_test
    self.y_test = y_test
    self.train_dataset = EEGDataset(X_train, y_train)
    self.test_dataset = EEGDataset(X_test, y_test)

###Fully-Connected Neural Network

In [0]:
class FCNN(nn.Module):
  pass

###Convolutional Neural Network

####CNN1
Extremely Vanilla CNN. Note the 3x3 convolutions do not take advantage of the temporal information.

conv+relu - conv+relu - pool - fc+relu - fc+relu - fc

In [0]:
class CNN1(nn.Module):
  def __init__(self):
    super(CNN1, self).__init__()
    self.conv1 = nn.Conv2d(1, 4, 3, stride=1, padding=1)
    self.conv2 = nn.Conv2d(4, 8, 3, stride=1, padding=1)
    self.pool = nn.MaxPool2d(2, 2)
    self.fc1 = nn.Linear(8 * 11 * 500, 120)
    self.fc2 = nn.Linear(120, 80)
    self.fc3 = nn.Linear(80, 4)
  
  def forward(self, x):
    # x is 1 x 22 x 1000
    x = self.pool(F.relu(self.conv2(F.relu(self.conv1(x)))))
    # x is 8 x 11 x  500
    x = x.view(-1, 8 * 11 * 500)
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.fc3(x)
    return x    

##Setup

In [0]:
data_dir = 'drive/ee239as/project_datasets'
batch_size = 17
num_epochs = 10
learning_rate = 1e-3

use_cuda = True

In [9]:
EEGset = EEGMinimalContainer(data_dir)

train_loader = torch.utils.data.DataLoader(EEGset.train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(EEGset.test_dataset,
                                          batch_size=10,
                                          shuffle=False)

EEGContainer X_train: (2108, 1, 22, 1000)
EEGContainer y_train: (2108,)
EEGContainer X_test:  (450, 1, 22, 1000)
EEGContainer y_test:  (450,)


In [0]:
net = CNN1()

if use_cuda and torch.cuda.is_available():
  net.cuda()

In [0]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)

##Training

In [12]:
for epoch in range(num_epochs):
  for i, (signals, labels) in enumerate(train_loader):
    
    signals = signals.type(torch.FloatTensor)
    signals = Variable(signals)
    labels = labels.type(torch.LongTensor)
    labels = Variable(torch.squeeze(labels))
    
    if use_cuda and torch.cuda.is_available():
      signals = signals.cuda()
      labels = labels.cuda()
    
    optimizer.zero_grad()
    outputs = net(signals)
    
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()
    
    if (i+1) % 31 == 0:
      print('Epoch [%d/%d], Step [%d/%d], Loss: %.4f' 
            % (epoch+1, num_epochs, i+1, len(EEGset.train_dataset)//batch_size, 
               loss.data[0]))
  
  # Training accuracy
  total = 0
  correct = 0
  for signals, labels in train_loader:
    signals = signals.type(torch.FloatTensor)
    signals = Variable(signals.view(batch_size, 1, 22, 1000))
    labels = torch.squeeze(labels.type(torch.LongTensor))
    if use_cuda and torch.cuda.is_available():
      signals = signals.cuda()
      labels = labels.cuda()
    outputs = net(signals)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum()
  print ('Training Accuracy: %.5f' % (correct/total))
  
  # Testing accuracy
  total = 0
  correct = 0
  for signals, labels in test_loader:
    signals = signals.type(torch.FloatTensor)
    signals = Variable(signals.view(10, 1, 22, 1000))
    labels = torch.squeeze(labels.type(torch.LongTensor))
    if use_cuda and torch.cuda.is_available():
      signals = signals.cuda()
      labels = labels.cuda()
    outputs = net(signals)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum()
  print ('Testing Accuracy: %.5f' % (correct/total))
    
      

Epoch [1/10], Step [31/124], Loss: 1.5956
Epoch [1/10], Step [62/124], Loss: 1.5133
Epoch [1/10], Step [93/124], Loss: 1.4063
Epoch [1/10], Step [124/124], Loss: 1.4249
Training Accuracy: 0.61006
Testing Accuracy: 0.41778
Epoch [2/10], Step [31/124], Loss: 1.0758
Epoch [2/10], Step [62/124], Loss: 1.0824
Epoch [2/10], Step [93/124], Loss: 1.1720
Epoch [2/10], Step [124/124], Loss: 1.3986
Training Accuracy: 0.61717
Testing Accuracy: 0.40667
Epoch [3/10], Step [31/124], Loss: 0.8280
Epoch [3/10], Step [62/124], Loss: 0.6866
Epoch [3/10], Step [93/124], Loss: 0.9393
Epoch [3/10], Step [124/124], Loss: 0.6384
Training Accuracy: 0.92979
Testing Accuracy: 0.45111
Epoch [4/10], Step [31/124], Loss: 0.2075
Epoch [4/10], Step [62/124], Loss: 0.1482
Epoch [4/10], Step [93/124], Loss: 0.3736
Epoch [4/10], Step [124/124], Loss: 0.1354
Training Accuracy: 0.98719
Testing Accuracy: 0.45778
Epoch [5/10], Step [31/124], Loss: 0.0633
Epoch [5/10], Step [62/124], Loss: 0.1022
Epoch [5/10], Step [93/124],